<a href="https://colab.research.google.com/github/spaziochirale/EsperimentiVari/blob/main/PrototipoRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prototipo di ChatBot RAG con LangChain

Questo notebook mostra come realizzare un *chatBot* di tipo **RAG** utilizzando il framework **LangChain**.

Per prima cosa installiamo sul server alcuni package Python della piattaforma LangChain che utilizzeremo nel prototipo.

***langchain*** è il modulo principale, ***langchain_community*** è il modulo che contiene i contributi di terze parti e della community al progetto LangChain e ***langchain_chroma*** è il modulo che contiene l'interfaccia verso il dtatabase vettoriale open source **Chroma**, usato in questo esempio.

Il punto esclamativo prima del comando serve a fare in modo che questo comando sia una istruzione eseguita dal Sistema Operativo Linux del server virtuale Google Colab. Normalmente le celle di un notebook come questo contengono istruzioni in linguaggio Python che vengono eseguite dall'interprete Python.

Pip è il **gestore dei pacchetti** di Python e serve per scaricare dai repository ufficiali le librerie e installarle sul server locale. Pip si esegue da linea di comando direttamente sul terminale del server. Il server virtuale creato da Google Colab non contiene le librerie di LangChain, nè altre librerie specifiche di terze parti, per cui vanno installate tramite l'utility **pip**.

In [ ]:
!pip install langchain langchain_community langchain_chroma

Se vogliamo utilizzare i LLM di OpenAI, dobbiamo installare il modulo specifico di LangChain che interfaccia le API di OpenAI.
L'istruzione pip che segue effettua questa installazione. le opzioni -qU eliminano l'output a video e forzano l'upgrade alla versione più aggiornata qualora il modulo fosse già aggiornato.

In [ ]:
!pip install -qU langchain-openai

Nella cella di codice che segue viene impostata sul server la variabile di ambiente OPENAI_API_KEY, il cui valore viene prelevato dalla sezione "**secrets**" del Colab. Per poter utilizzare questo notebook, occorre quindi creare un secret colab di etichetta OPENAI_API_KEY e valorizzarla con una chiave OpenAI valida.

Successivamente viene creato un oggetto chiamato **llm** che rappresenta l'interfaccia LangChain verso il modello **gpt-4o-mini** di OpenaAI.

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

La cella successiva contiene le istruzioni import per importare nel nostro programma tutte le librerie Python che saranno utilizzate.

- bs4 è una nota libreria open source, chiamata Beautiful Soup, che facilita le operazioni di web scraping e parsing di pagine HTML dai siti web.
- hub è un modulo di LangChain che contiene risorse ed esempi già pronti che velocizzano lo sviluppo. Nel nostro programma servirà per recuperare un template di prompt per il LLM, ben strutturato per il RAG.
- Chroma è un database vettoriale open source molto leggero e flessibile.
- WebBaseLoaderè un modulo LangChain, di tipo document loader, sviluppato dalla community per importare pagine dal web.
- StrOutParser è un modulo di langChain per semplificare le operazioni di parsing su stringhe complesse
- RunnablePassthrough è un modulo LangChain che automatizza il passaggio di valori e parametri tra i passi contigui in una chain. Si veda la documentazione LangChain per dettagli su questi concetti.
- OpenAIEmbeddings è il modulo LangChain di interfaccia alle API Embeddings di OpenAI
- RecursiveCharacterTextSplitter è un modulo LangChain per effettuare l'operazione di spezzettamento in chunck di stringhe di testo corrispondenti a testi molto lunghi.

In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


Il frammento di codice che segue effettua lo scraping di una pagina web.
La pagina catturata da Internet viene trasformata in una grande e unica stringa di testo posta nella variabile docs.
Un loader LangChain normalmente crea una lista di documenti di tipo testo. La variabile docs è quindi una lista Python ma nel nostro caso contiene un solo elemento.

In [ ]:

loader = WebBaseLoader(
    web_paths=("https://www.chirale.it/fotocamere-antiche-come-costruire-con-arduino-un-misuratore-di-velocita-dellotturatore/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()



Proviamo ad esplorare quello che è successo.
Vediamo qual'è il tipo della variabile docs e stampiamo il suo contenuto.

In [ ]:
# L'esecuzione di questa cella è opzionale
print('Tipo della variabile:',type(docs))
print(docs)

Il codice della cella che segue crea un oggetto, chiamato **text_splitter** di tipo ***RecursiveCharacterTextSplitter*** con dimensioni dei chunk pari a 1.000 caratteri e una sovrapposizione tra chunk di 200 caratteri.

La stringa contenuta nella variabile **`docs`** viene spezzata in chunks usando lo splitter definito e il risultato, cioè la lista dei singoli frammenti di testo (chunk) è memorizzata nella variabile **splits**.

Infine, viene creato l'oggetto **vectorstore** come database Chroma contenente i chunk vettorializzati tramite l'embedding di OpenAI e indicizzati.
Il metodo **Chroma.from_documents** di LangChain fa esattamente questo con una sola chiamata.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())



Anche in questo caso proviamo a vedere il contenuto della variabile splits ed esploriamo il suo tipo e la sua struttura.

In [ ]:
# L'esecuzione di questa cella di codice è opzionale
print('Tipo della variabile:',type(splits))
print('lunghezza di splits:', len(splits))
print(splits[0])
print(splits[1])

Nella cella seguente, viene creato l'oggetto **retriever** invocando il metodo **as_retriever()** dell'oggetto **vectorstore**.
Questa è un'altra delle caratteristiche che mostrano la potenza di LangChain. In LangChain gli oggetti che rappresentano l'interfaccia verso un database vettoriale possono essere utilizzati anche come ***retriever***.
I *retriever* LangChain sono degli oggetti specializzati nel recupero di testo mediante una ricerca semantica sul database vettoriale.
Semplificano il passaggio di una stringa contenente la *query*, la sua vettorializzazione mediante *embeddings* e il recupero dei chunk di testo più pertinenti dal database vettoriale.

Successivamente viene creato l'oggetto **prompt** contenente il ***prompt template*** adatto al nostro caso, recuperato dall'hub di LangChain.

I *prompt template* sono un'altra delle potenzialità offerta da LangChain. Si veda la documentazione.

In [ ]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


Nel seguito viene definita una semplice funzione di utilità per formattare come stringa multiriga una sequenza di testi.

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


Ed ecco, la classica definizione della chain di LangChain, dove attraverso il suo formalismo *LCEL*, viene definita la sequenza di operazioni da effettuare.
Il primo elemento della chain è un dizionario Python in cui alla chiave "**context**" viene associato il risultato di una catena di due operazioni: il valore dell'oggetto **retriever** formattato dalla funzione di utilità che abbiamo definito poco sopra.
Alla chiave "**question**" viene associato il valore restituito dal metodo **RunnablePassThgrough**, cioè il valore inserito quando la chain sarà invocata.
Questo dizionario sarà passato al *prompt template*, **prompt**, che quindi definirà il prompt effettivo, formattato in modo corretto.
Tale prompt viene quindi passato all'oggetto **llm**, che invocherà le API di Chat di OpenAI e l'output risultante sarà passato al metodo ***StrOutputParser*** che estrarrà il messaggio di risposta.


In [ ]:

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


A questo punto possiamo invocare la chain rag_chain passando la query. Il sistema risponderà secondo il contenuto del nostro sito web.

In [ ]:

rag_chain.invoke("Dove posso acquistare oggigiorno lastre fotografiche in vetro?")